In [ ]:
#require "pkp"
#print_length 10

In [ ]:
open Owl
open Gp

In [ ]:
let _ = Pkp.Misc.quiet_owl ()

## Single spike trains

Let's start with the [exponential distribution](https://www.wikiwand.com/en/Exponential_distribution):

In [ ]:
let data = Mat.init 1 10000 (fun _ -> Owl_stats.exponential_rvs ~lambda:10.)

In [ ]:
let _ =
  let h = Pkp.Misc.hist ~n_bins:50 data in
  let fig (module P : Plot) =
    P.plot
      (A h)
      ~style:"boxes fs solid 0.2 lc 8"
      [ barebone
      ; borders [ `bottom ]
      ; xlabel "variable"
      ; ylabel "density"
      ; xrange (-0.02, 0.8)
      ; xtics `auto
      ]
  in
  Juplot.draw ~size:(300, 200) fig

A "Poisson process" (also called a "Poisson spike train") is a sequences of events, with consecutive events separated by a time interval that is exponentially distributed:

In [ ]:
let poisson_spike_train ~rate duration =
  let rec build accu t =
    let new_t = t +. Owl_stats.exponential_rvs ~lambda:rate in
    if new_t > duration then List.rev accu else build (new_t :: accu) new_t
  in
  build [] 0.

In [ ]:
let plot_spike_train train =
  let x = Mat.of_array (Array.of_list train) 1 (-1) in
  let fig (module P : Plot) =
    P.plot
      (A x)
      ~using:"1:(1)"
      ~style:"impulses lc 8"
      [ barebone; borders [ `bottom ]; xtics `auto; xlabel "time (s)" ]
  in
  Juplot.draw ~size:(600, 200) fig

In [ ]:
let _ = poisson_spike_train ~rate:5. 2. |> plot_spike_train

These are called "Poisson spike trains" because, if we take a time window of size $T$ (say, 1 sec), and count the number of spikes that fall in that window, we get an integer random number that follows the so-called [Poisson distribution](https://www.wikiwand.com/en/Poisson_distribution):

In [ ]:
let counts =
  let window = 1.0 in
  Mat.init 1 10000 (fun _ ->
      let spike_train = poisson_spike_train ~rate:10. window in
      float (List.length spike_train))

In [ ]:
let _ =
  let n_bins = int_of_float (Mat.max' counts) in
  let h = Pkp.Misc.hist ~n_bins counts in
  let fig (module P : Plot) =
    P.plot
      (A h)
      ~style:"boxes fs solid 0.2 lc 8"
      [ barebone
      ; borders [ `bottom ]
      ; set "xrange [-0.5:]"
      ; xlabel "spike count"
      ; ylabel "density"
      ; xtics `auto
      ]
  in
  Juplot.draw ~size:(300, 200) fig

This was what we call an "homogeneous Poisson process": one for which the rate of action potential emission is constant in time. You can also make a spike train with a time-varying rate:

In [ ]:
let inhomogeneous_poisson_spike_train ~rate duration =
  let dt = 1E-4 in
  let rec iter accu k =
    let t = dt *. float k in
    if t > duration
    then List.rev accu
    else (
      let accu = if Random.float 1.0 < dt *. rate t then t :: accu else accu in
      iter accu (k + 1))
  in
  iter [] 0

In [ ]:
let _ =
  let rate t = 100. *. (1. +. sin (2. *. Const.pi *. t)) /. 2. in
  inhomogeneous_poisson_spike_train ~rate 5.0 |> plot_spike_train

The so-called “firing rate” is the average number of spikes per second:

In [ ]:
let check_rate desired =
  let duration = 1000000. in
  let train = poisson_spike_train ~rate:desired duration in
  float (List.length train) /. duration

In [ ]:
check_rate 10.4

The **Fano factor** is a measure of spiking irregularity / variability. Imagine sliding a window of size, say, 100 ms, over a very long spike train, and counting the number of action potentials each time. You get a collection of spike counts. The Fano factor is the variance of that collection, divided by its mean. For a Poisson process with constant rate, the Fano factor is know to be exactly one:

In [ ]:
let _ = Pkp.Balanced_net.fano_factor ~window:0.1 (poisson_spike_train ~rate:5. 100.)

This is close to what is typically measured in cortical neurons (though it's usually slightly above one ─ this will be discussed in lectures). 

## Population spike trains

In [ ]:
let plot_spike_trains trains =
  let trains = Array.map (fun x -> Mat.of_array (Array.of_list x) 1 (-1)) trains in
  let t_min = trains |> Array.map Mat.min' |> Array.fold_left min max_float in
  let t_max = trains |> Array.map Mat.max' |> Array.fold_left max (-.max_float) in
  let n = Array.length trains in
  let fig (module P : Plot) =
    P.multiplot
      (n, 1)
      ~rect:((0.05, 0.2), (0.95, 0.95))
      ~spacing:(0., 0.002)
      (fun k _ _ ->
        let x = trains.(k) in
        let props =
          [ barebone; unset "key"; xrange (t_min, t_max) ]
          @
          if k < n - 1
          then []
          else [ borders [ `bottom ]; xtics (`regular [ 0.; 1. ]); xlabel "time (s)" ]
        in
        P.plot (A x) ~using:"1:(1)" ~style:"impulses lc 8 lw 2" props)
  in
  Juplot.draw ~size:(600, 400) fig

In [ ]:
let _ = Array.init 20 (fun _ -> poisson_spike_train ~rate:4. 2.) |> plot_spike_trains

In [ ]:
let _ =
  let rate t = 10. *. (1. +. sin (2. *. Const.pi *. t)) /. 2. in
  Array.init 200 (fun _ -> inhomogeneous_poisson_spike_train ~rate 3.)
  |> plot_spike_trains

The so-called “population firing rate” $r(t)$ is the momentary average spike rate across the population. In any small time window of length $\Delta_t$ starting at time $t$, you can measure the fraction of neurons in the population that have spiked in that window, and that's basically $r(t)\Delta_t$.

In [ ]:
let population_rate ~delta_t trains =
  let n = Array.length trains in
  let t_min, t_max =
    let f op a = trains |> Array.map (List.fold_left op a) |> Array.fold_left op a in
    f min max_float, f max (-.max_float)
  in
  let t_max = trains |> Array.map (List.fold_left max 0.) |> Array.fold_left max 0. in
  let rec iter accu t =
    if t > t_max
    then accu |> List.rev |> Array.of_list |> Mat.of_arrays
    else (
      let t' = t +. delta_t in
      let count =
        Array.fold_left
          (fun accu train ->
            if List.exists (fun t_spike -> t_spike >= t && t_spike < t') train
            then accu + 1
            else accu)
          0
          trains
      in
      let accu = [| t +. (delta_t /. 2.); float count /. float n /. delta_t |] :: accu in
      iter accu t')
  in
  iter [] t_min

In [ ]:
let _ =
  let rate t = 10. *. (1. +. sin (2. *. Const.pi *. t)) /. 2. in
  let duration = 3. in
  let trains =
    Array.init 1000 (fun _ -> inhomogeneous_poisson_spike_train ~rate duration)
  in
  let r = population_rate ~delta_t:1E-3 trains in
  let fig (module P : Plot) =
    P.plots
      [ item (A r) ~style:"l lc 8"
      ; item (F (rate, Mat.linspace 0. duration 100)) ~style:"l lc 7 lw 3"
      ]
      [ borders [ `bottom; `left ]
      ; xtics (`regular [ 0.; 1. ])
      ; xlabel "time (s)"
      ; ylabel "population rate (Hz)"
      ]
  in
  Juplot.draw ~size:(400, 200) fig